# Utilizando banco de dados vetorial

Neste notebook iremos utilizar uma banco de dados vetorial para armazenar nossos vetores de maneira mais eficiente do que utilizando o SqLite.

Existem diveras opções de bancos vetoriais para serem utilizadas:

![Vector Database Examples](../public/vecdb-examples.png)

Utilizaremos o ChromaDB para o nosso exemplo. O ChromaDB, assim como o Sqlite, é um banco de dados de ficheiro, ou seja, um banco de dados que pode ser armazenado localmente.



## 1 - Configuração

In [1]:
# Recarrega automaticamente quando há modificação na pasta src/
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

import pandas as pd

In [3]:
# Configuração de diretórios
HERE = os.path.abspath(".")
MODULES = Path(HERE).parent
DATA = Path(HERE).parent / "data"

sys.path.insert(
    0, str(MODULES)
)  # para ser possível importar os módulos personalizados da pasta src/

## 2 - ETL

Construíremos um ETL para fazer algumas transformações no nosso banco de dados para prepara-lo para popular o vector database.


In [4]:
pd.read_csv(DATA / "0_raw" / "medium_post_titles.csv").head()

,category,title,subtitle,subtitle_truncated_flag
0,work,"""21 Conversations"" - A fun (and easy) game for...",A (new?) Icebreaker game to get your team to s...,False
1,spirituality,"""Biblical Porn"" at Mars Hill",Author and UW lecturer Jessica Johnson talks a...,False
2,lgbtqia,"""CISGENDER?! Is That A Disease?!""","Or, a primer in gender vocabulary for the curi...",False
3,equality,"""Call me Nat Love"" :Black Cowboys and the Fron...",NaN,False
4,artificial-intelligence,"""Can I Train my Model on Your Computer?""",How we waste computational resources and how t...,False


O nosso ETL consiste nos passos:

1. Retirar NA's
1. Remover linhas que possuem a flag `subtitle_truncated_flag`.
1. Filtrar as categorias para: "artificial-intelligence", "data-science" e "machine-learning".
1. Criar a coluna "Text" concatenando title e subtitle. Essa coluna será a que utilizaremos para construir os nosso vetores
1. Criando coluna "Metadata" contendo informações que correspondem ao nosso vetor
1. Criando coluna de ID
1. Filtrando as colunas para as que utilizaremos ao popular o vector database: "id", "text" e "metadata"

In [ ]:
from src.pipe import ETL

etl = ETL(DATA / "0_raw" / "medium_post_titles.csv")

etl.pipeline()

## 3 - Criando o vector database

### 3.1 - Criando a conexão com o ChromaDB

In [6]:
from src.io import ChromaDBIO

chroma = ChromaDBIO(
    DATA / "2_processed" / "medium-chroma-db" / "medium-article"
)

### 3.2 - Populando o ChromaDB

A vantagem de utilizar um banco de dados vetorial é que estes já incorporam modelos de embedding durante o processo de população dos dados. Isso faz com que o nosso script fique limpo, leve e rápido - principalmente quando o banco de dados vetorial está em núvem.

Por padrão o ChromaDB utiliza o all-MiniLM-L6-v2 para fazer o embedding dos vetores. Porém é possível incorporar outros modelos que estão disponíveis na biblioteca, modelos do HuggingFace e até mesmo da OpenAI. O ChromaDB também disponibiliza integração com o LangChain.


In [7]:
chroma.upsert_data("medium_post_titles.parquet")

O ChromaDB suporta as seguintes métricas de similaridade:

Distância -> Parâmetro -> Equação

Squared L2 -> "l2" -> $d = \sum\left(A_i-B_i\right)^2$

Inner Product -> "ip" -> $d = 1.0 - \sum\left(A_i \times B_i\right)$

Cosine Similarity -> "cosine" -> $d = 1.0 - \frac{\sum\left(A_i \times B_i\right)}{\sqrt{\sum\left(A_i^2\right)} \cdot \sqrt{\sum\left(B_i^2\right)}}$

Por padrão o Chroma utiliza a distância L2.

In [8]:
chroma.query("Data Science", n_results=1).model_dump()

{'ids': [['493']],
 'distances': [[0.7876980230561456]],
 'metadatas': [[{'category': 'data-science',
    'text': '10 Machine Learning Methods that Every Data Scientist Should Know Jump-start your data science skills'}]],
 'embeddings': None,
 'documents': [['10 Machine Learning Methods that Every Data Scientist Should Know Jump-start your data science skills']]}

In [9]:
chroma.read_to_pandas("Data Science", n_results=2)

,ids,distances,documents,category,text
1,289,0.841722,(Robot) data scientists as a service Automatin...,data-science,(Robot) data scientists as a service Automatin...
0,493,0.787698,10 Machine Learning Methods that Every Data Sc...,data-science,10 Machine Learning Methods that Every Data Sc...
